# Olist Dataset

This notebook is used to prepare the olist dataset for ingestion into the streamlit app, to apply customer segmentation and customer lifetime value (CLV) prediction.

The dataset can be found here: *https://www.kaggle.com/datasets/olistbr/brazilian-ecommerce?resource=download*

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from datetime import datetime, timedelta, date

## Data Ingestion

In [2]:
orders_dataset = pd.read_csv('../data/olist_orders_dataset.csv')
orders_dataset = orders_dataset[["order_id", "customer_id", "order_purchase_timestamp"]]

In [3]:
orders_dataset.head(2)

,order_id,customer_id,order_purchase_timestamp
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37


In [4]:
payments_dataset = pd.read_csv('../data/olist_order_payments_dataset.csv')
payments_dataset = payments_dataset[["order_id", "payment_value"]]

In [5]:
payments_dataset.head(2)

,order_id,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,24.39


## Data Cleaning

In [6]:
## merge datasets
dataframe = pd.merge(orders_dataset, payments_dataset, on= "order_id", how = "inner")

In [7]:
dataframe.drop("order_id", axis= 1, inplace= True)

In [8]:
dataframe.rename({"customer_id": "CustomerID",
                 "order_purchase_timestamp":"InvoiceDate",
                 "payment_value": "Revenue"}, 
                 axis=1,
                 inplace=True)

In [9]:
dataframe["InvoiceDate"] = pd.to_datetime(dataframe["InvoiceDate"]).dt.date

In [10]:
dataframe.head()

,CustomerID,InvoiceDate,Revenue
0,9ef432eb6251297304e76186b10a928d,2017-10-02,18.12
1,9ef432eb6251297304e76186b10a928d,2017-10-02,2.00
2,9ef432eb6251297304e76186b10a928d,2017-10-02,18.59
3,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24,141.46
4,41ce2a54c0b03bf3443c3d931a367089,2018-08-08,179.12


In [11]:
dataframe.shape

(103886, 3)

In [12]:
dataframe.CustomerID.nunique()

99440

In [13]:
dataframe.dtypes

CustomerID      object
InvoiceDate     object
Revenue        float64
dtype: object

In [14]:
## assessing how many customers are repeat customers
_ = pd.DataFrame(dataframe.groupby("CustomerID")["InvoiceDate"].count())

repeat_cust = _.loc[_["InvoiceDate"] > 1].shape[0]

print(f"{round(repeat_cust / _.shape[0],2)}% repeat customers")

0.03% repeat customers


As there are only <1% repeat customers, the analysis will proceed with only those customers who have shopped at the Olist online store repeatedly.

In [15]:
df_tmp = pd.DataFrame(dataframe.groupby("CustomerID")["InvoiceDate"].count()).reset_index()

df_new = pd.merge(df_tmp, dataframe, on= "CustomerID")

In [16]:
df_new.rename({"InvoiceDate_x": "BuyCount",
              "InvoiceDate_y": "InvoiceDate"}, axis= 1, inplace= True)

In [17]:
final_df = df_new.loc[df_new["BuyCount"] > 1]

In [18]:
final_df.columns

Index(['CustomerID', 'BuyCount', 'InvoiceDate', 'Revenue'], dtype='object')

In [19]:
final_df.CustomerID.nunique()

2961

In [20]:
final_df.head()

,CustomerID,BuyCount,InvoiceDate,Revenue
16,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
17,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
18,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
19,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,25.83
25,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,13.35


In [21]:
## assessing total revenue by repeat customers
revenue = round(final_df.Revenue.sum(),2)

total_revenue = round(dataframe.Revenue.sum(),2)

perc = round(revenue/total_revenue,2)

print(f"Repeat customers make up {revenue}.\n\nThat is {perc}% of total revenue.")

Repeat customers make up 492107.95.

That is 0.03% of total revenue.


## Export Final Dataset

In [22]:
final_df.to_csv("../data/final_olist_dataset.csv", index= False)

# CLV Prediction

## Creating RFM Metrics

In [23]:
import sys
sys.path.append('../')

In [24]:
from modules.cleaning import clean_dataframe
from modules.features import clustering, order_clusters

In [25]:
final_df = clean_dataframe(final_df);

/Users/garritwilson/Desktop/Portfolio Projects/clv_prediction/notebooks/../modules/cleaning.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [26]:
print(final_df.InvoiceDate.min(), final_df.InvoiceDate.max())

2016-10-04 00:00:00 2018-08-28 00:00:00


In [27]:
final_df.head()

,CustomerID,BuyCount,InvoiceDate,Revenue
16,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
17,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
18,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,26.80
19,000e943451fc2788ca6ac98a682f2f49,4,2017-04-20,25.83
25,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,13.35


In [28]:
data = final_df.copy()

In [29]:
## making 3 - months and 6 - months dataframes
data_3m = data[(data.InvoiceDate.dt.date < date(2018,2,28))].reset_index()
data_6m = data[(data.InvoiceDate.dt.date >= date(2018,2,28))].reset_index()

In [30]:
data_6m.head()

,index,CustomerID,BuyCount,InvoiceDate,Revenue
0,25,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,13.35
1,26,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,19.82
2,27,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,32.03
3,196,007b18ac9b8a627f259ea78aed981315,2,2018-03-16,41.13
4,197,007b18ac9b8a627f259ea78aed981315,2,2018-03-16,2.00


In [31]:
data.shape

(7407, 4)

In [32]:
dataframe = data_3m.copy()

In [33]:
user_df = pd.DataFrame(dataframe.CustomerID.unique(), columns= ["CustomerID"])

## creating Recency Metric
recency_df = pd.DataFrame(dataframe.groupby("CustomerID")["InvoiceDate"].max().reset_index())
recency_df.columns = ["CustomerID", "LatestPurchase"]

recency_df["Recency"] = (dataframe["InvoiceDate"].max() - recency_df["LatestPurchase"]).dt.days
recency_df.drop("LatestPurchase", axis= 1, inplace= True)

recency_df = clustering(data= recency_df,
                        k= 3,
                        column="Recency")

recency_df = order_clusters(data= recency_df,
                            column= "RecencyCluster",
                            target= "Recency",
                            ascending= False)

user_df = pd.merge(recency_df, user_df, on= "CustomerID") 

## creating Frequency Metric
frequency_df = pd.DataFrame(dataframe.groupby("CustomerID")["InvoiceDate"].count().reset_index())
frequency_df.columns = ["CustomerID", "Frequency"]

frequency_df = clustering(data= frequency_df,
                          k= 5,
                          column= "Frequency")

frequency_df = order_clusters(data= frequency_df,
                              column= "FrequencyCluster",
                              target= "Frequency",
                              ascending= True)

user_df = pd.merge(frequency_df, user_df, on= "CustomerID")

## creating Revenue Metric
revenue_df = pd.DataFrame(dataframe.groupby("CustomerID")["Revenue"].sum().reset_index())
revenue_df.columns = ["CustomerID", "Revenue"]

revenue_df = clustering(data= revenue_df,
                        k= 5,
                        column= "Revenue")

revenue_df = order_clusters(data= revenue_df,
                            column= "RevenueCluster",
                            target= "Revenue",
                            ascending= True)

user_df = pd.merge(revenue_df, user_df, on= "CustomerID")

user_df["OverallScore"] = user_df["RecencyCluster"] + user_df["FrequencyCluster"] + user_df["RevenueCluster"]

In [34]:
user_df.head()

,CustomerID,Revenue,RevenueCluster,Frequency,FrequencyCluster,Recency,RecencyCluster,OverallScore
0,000e943451fc2788ca6ac98a682f2f49,106.23,0,4,4,313,2,6
1,004937d0f9d6ce15c2830c00c2f482e5,70.91,0,2,0,31,0,0
2,006a5d6b5f648f3811fd4fa94d93a67c,76.94,0,2,0,273,2,2
3,008def95c3ec7f58d2cc8f2b1d2c3feb,51.14,0,2,0,190,1,1
4,00a860aeb15fb205efdb0c689d8f9c6a,73.34,0,2,0,175,1,1


In [35]:
data_6m.head()

,index,CustomerID,BuyCount,InvoiceDate,Revenue
0,25,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,13.35
1,26,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,19.82
2,27,001051abfcfdbed9f87b4266213a5df1,3,2018-05-30,32.03
3,196,007b18ac9b8a627f259ea78aed981315,2,2018-03-16,41.13
4,197,007b18ac9b8a627f259ea78aed981315,2,2018-03-16,2.00


In [36]:
df_merged = pd.merge(user_df, data_6m, on= "CustomerID", how= "left")

In [37]:
df_merged.head()

,CustomerID,Revenue_x,RevenueCluster,Frequency,FrequencyCluster,Recency,RecencyCluster,OverallScore,index,BuyCount,InvoiceDate,Revenue_y
0,000e943451fc2788ca6ac98a682f2f49,106.23,0,4,4,313,2,6,NaN,NaN,NaT,NaN
1,004937d0f9d6ce15c2830c00c2f482e5,70.91,0,2,0,31,0,0,NaN,NaN,NaT,NaN
2,006a5d6b5f648f3811fd4fa94d93a67c,76.94,0,2,0,273,2,2,NaN,NaN,NaT,NaN
3,008def95c3ec7f58d2cc8f2b1d2c3feb,51.14,0,2,0,190,1,1,NaN,NaN,NaT,NaN
4,00a860aeb15fb205efdb0c689d8f9c6a,73.34,0,2,0,175,1,1,NaN,NaN,NaT,NaN
